In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape text from a given URL
def scrape_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find and extract text content from relevant elements on the webpage
    text = ""
    for paragraph in soup.find_all('p'):
        text += paragraph.get_text() + "\n"
    return text.strip()

# Function to create the dataset
def create_dataset(urls, scores):
    texts = []
    for url in urls:
        texts.append(scrape_text(url))
    # Create DataFrame with text and scores
    df = pd.DataFrame({'Text (Arabic Language)': texts, 'Score': scores})
    return df

# URLs of Arabic news websites related to technology
urls = [
    'https://www.tech-wd.com/wd/2024/04/03/%d8%aa%d8%b3%d8%b1%d9%8a%d8%a8%d8%a7%d8%aa-%d8%aa%d8%b5%d9%85%d9%8a%d9%85-%d8%aa%d8%b7%d8%a8%d9%8a%d9%82-%d8%a7%d9%84%d9%83%d8%a7%d9%85%d9%8a%d8%b1%d8%a7-ios-18/',
    'https://www.elfann.com/news/show/1367468/%D8%A8%D8%B9%D8%AF-%D8%BA%D9%8A%D8%A7%D8%A8%D9%8D-%D8%A7%D9%84%D8%A3%D9%85%D9%8A%D8%B1%D8%A9-%D8%B4%D8%A7%D8%B1%D9%84%D9%8A%D9%86-%D9%85%D9%88%D9%86%D8%A7%D9%83%D9%88-%D8%AA%D8%AA%D8%A3%D9%84%D9%91%D9%82-%D8%A8%D8%AC%D9%85%D8%A8%D8%B3%D9%88%D8%AA-%D8%A5%D9%8A%D9%84',
 ' https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%84%D8%BA%D8%A9_%D8%A7%D9%84%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9',
   ' https://www.alarabiya.net/',
    'https://alarab.co.uk/',
    'https://www.dw.com/ar/%D8%A7%D9%84%D8%B1%D8%A6%D9%8A%D8%B3%D9%8A%D8%A9/s-9106',
    'https://www.araby.ai/',
    'https://www.france24.com/ar/',
    'https://www.skynewsarabia.com/'

]

# Relevance scores for each text (should be between 0 to 10)
scores = [10]* len(urls)

# Create the dataset
dataset = create_dataset(urls, scores)

# Save the dataset to a CSV file
dataset.to_csv('arabic_tech_news_dataset.csv', index=False, encoding='utf-8-sig')

print("Dataset created and saved successfully.")


Dataset created and saved successfully.


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Tokenization
        tokens = word_tokenize(text)

        # Remove stop words
        stop_words = set(stopwords.words('arabic'))
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

        # Stemming
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

        # Join tokens back into a single string
        preprocessed_text = ' '.join(lemmatized_tokens)

        return preprocessed_text
    else:
        return ''  # Return an empty string for non-string inputs


print("Preprocessing completed and preprocessed dataset saved successfully.")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Preprocessing completed and preprocessed dataset saved successfully.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('arabic'))
        filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
        preprocessed_text = ' '.join(lemmatized_tokens)
        return preprocessed_text
    else:
        return ''

# Step 1: Prepare your dataset
# Load the preprocessed dataset
dataset = pd.read_csv('arabic_tech_news_dataset.csv')

# Step 2: Preprocess the text data
dataset['Preprocessed Text'] = dataset['Text (Arabic Language)'].apply(preprocess_text)

# Step 3: Encode the text data (if necessary)

# Step 4: Split the dataset into training, validation, and testing sets
train_texts, test_texts, train_scores, test_scores = train_test_split(dataset['Preprocessed Text'],
                                                                      dataset['Score'],
                                                                      test_size=0.2,
                                                                      random_state=42)
train_texts, val_texts, train_scores, val_scores = train_test_split(train_texts,
                                                                    train_scores,
                                                                    test_size=0.2,
                                                                    random_state=42)

# Step 5: Define model architectures
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

class BidirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BidirectionalRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Step 6: Train the models
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                val_loss += loss.item()
            val_loss /= len(val_loader)

        print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# Step 7: Evaluate the models
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            test_loss += loss.item()
        test_loss /= len(test_loader)
    return test_loss


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

# Step 8: Prepare DataLoader for training, validation, and testing sets
class CustomDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.scores[idx]

def collate_fn(batch):
    texts, scores = zip(*batch)
    texts_tensor = torch.tensor(texts)  # Convert texts to tensor
    scores_tensor = torch.tensor(scores, dtype=torch.float32).unsqueeze(1)  # Convert scores to tensor and adjust shape
    return texts_tensor, scores_tensor



train_dataset = CustomDataset(train_texts, train_scores)
val_dataset = CustomDataset(val_texts, val_scores)
test_dataset = CustomDataset(test_texts, test_scores)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Step 9: Initialize and train the models
input_size = 300  # Input size should match with the vectorized dimension of the text data
output_size = 1  # Output size is 1 as we are predicting a continuous variable (score)

# Define hyperparameters
hidden_size = 128
learning_rate = 0.001
num_epochs = 10

# Initialize models
rnn_model = SimpleRNN(input_size, hidden_size, output_size)
bidirectional_rnn_model = BidirectionalRNN(input_size, hidden_size, output_size)
gru_model = GRUModel(input_size, hidden_size, output_size)
lstm_model = LSTMModel(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
rnn_optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)
bidirectional_rnn_optimizer = optim.Adam(bidirectional_rnn_model.parameters(), lr=learning_rate)
gru_optimizer = optim.Adam(gru_model.parameters(), lr=learning_rate)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

# Train the models
print("Training Simple RNN...")
train_model(rnn_model, train_loader, val_loader, criterion, rnn_optimizer, num_epochs=num_epochs)

print("Training Bidirectional RNN...")
train_model(bidirectional_rnn_model, train_loader, val_loader, criterion, bidirectional_rnn_optimizer, num_epochs=num_epochs)

print("Training GRU...")
train_model(gru_model, train_loader, val_loader, criterion, gru_optimizer, num_epochs=num_epochs)

print("Training LSTM...")
train_model(lstm_model, train_loader, val_loader, criterion, lstm_optimizer, num_epochs=num_epochs)

# Step 10: Evaluate the models
print("Evaluating Simple RNN...")
test_loss_rnn = evaluate_model(rnn_model, test_loader, criterion)
print(f"Test Loss (Simple RNN): {test_loss_rnn:.4f}")

print("Evaluating Bidirectional RNN...")
test_loss_bidirectional_rnn = evaluate_model(bidirectional_rnn_model, test_loader, criterion)
print(f"Test Loss (Bidirectional RNN): {test_loss_bidirectional_rnn:.4f}")

print("Evaluating GRU...")
test_loss_gru = evaluate_model(gru_model, test_loader, criterion)
print(f"Test Loss (GRU): {test_loss_gru:.4f}")

print("Evaluating LSTM...")
test_loss_lstm = evaluate_model(lstm_model, test_loader, criterion)
print(f"Test Loss (LSTM): {test_loss_lstm:.4f}")



Training Simple RNN...


KeyError: 1